In [ ]:
!pip install dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 37.2 MB/s eta 0:00:00


In [7]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import folium
import pandas as pd
import math

# Load the CSV file
data = pd.read_csv('thetest.csv')

# Drop rows with NaN values in the 'Cluster' column and convert to integers
data = data.dropna(subset=['Cluster'])
data['Cluster'] = data['Cluster'].astype(int)

# Adjust cluster indexing and extract necessary columns for plotting
locations = data[['Hour', 'Actual_AvgLat', 'Actual_AvgLon', 'Pred Avg Lat', 'Pred Avg Long', 'Cluster',
                  'Size_Threat_Score', 'IntraCluster_Threat_Score', 'Overall_Threat_Score', 'Distance_Threat_Score']].dropna()

# Define colors for clusters (actual and predicted)
actual_cluster_colors = {
    1: 'blue', 2: 'green', 3: 'purple', 4: 'orange', 5: 'darkred',
    6: 'lightred', 7: 'beige', 8: 'darkblue', 9: 'darkgreen', 10: 'cadetblue',
    11: 'pink', 12: 'lightblue', 13: 'lightgreen', 14: 'gray', 15: 'black', 16: 'lightgray'
}

predicted_cluster_colors = {
    1: 'navy', 2: 'lime', 3: 'magenta', 4: 'yellow', 5: 'firebrick',
    6: 'salmon', 7: 'khaki', 8: 'darkcyan', 9: 'forestgreen', 10: 'teal',
    11: 'hotpink', 12: 'skyblue', 13: 'yellowgreen', 14: 'dimgray', 15: 'maroon', 16: 'silver'
}

# Function to create a Folium map
def create_map(hour, show_actual_clusters, show_predicted_clusters, selected_threat_level):
    actual_locations = locations[(locations['Hour'] <= hour)]

    # Filter by selected threat level
    if selected_threat_level and selected_threat_level != "All":
        if selected_threat_level == "Low":
            actual_locations = actual_locations[actual_locations['Overall_Threat_Score'] <= 0.5]
        elif selected_threat_level == "Medium":
            actual_locations = actual_locations[(actual_locations['Overall_Threat_Score'] > 0.5) & (actual_locations['Overall_Threat_Score'] <= 0.8)]
        elif selected_threat_level == "High":
            actual_locations = actual_locations[actual_locations['Overall_Threat_Score'] > 0.8]

    if not actual_locations.empty:
        map_center = [actual_locations['Actual_AvgLat'].mean(), actual_locations['Actual_AvgLon'].mean()]
    else:
        map_center = [locations['Actual_AvgLat'].mean(), locations['Actual_AvgLon'].mean()]

    m = folium.Map(location=map_center, zoom_start=6)

    # Plot actual points for selected clusters up to and including the current hour
    for i in range(len(actual_locations)):
        cluster_idx = int(actual_locations.iloc[i]['Cluster'])
        if cluster_idx in show_actual_clusters:
            folium.Circle(
                location=(actual_locations.iloc[i]['Actual_AvgLat'], actual_locations.iloc[i]['Actual_AvgLon']),
                radius=300,
                color=actual_cluster_colors[cluster_idx],
                fill=True,
                fill_opacity=0.8
            ).add_to(m)

    # Plot predicted points for selected clusters
    for cluster_idx in show_predicted_clusters:
        cluster_data = locations[(locations['Cluster'] == cluster_idx)]
        for i in range(len(cluster_data)):
            folium.Circle(
                location=(cluster_data.iloc[i]['Pred Avg Lat'], cluster_data.iloc[i]['Pred Avg Long']),
                radius=200,
                color=predicted_cluster_colors[cluster_idx],
            ).add_to(m)

        # Add arrows connecting predicted points in sequence within the same cluster
        cluster_data_sorted = cluster_data.sort_values(by='Hour')
        coordinates = cluster_data_sorted[['Pred Avg Lat', 'Pred Avg Long']].values.tolist()

        if len(coordinates) > 1:
            folium.PolyLine(coordinates, color='red', weight=2, opacity=0.6).add_to(m)

            # Add arrow at the start and end points
            start = coordinates[0]
            end = coordinates[-1]

            start_angle = calculate_angle(start[0], start[1], coordinates[1][0], coordinates[1][1])
            end_angle = calculate_angle(coordinates[-2][0], coordinates[-2][1], end[0], end[1])

            folium.Marker(
                location=start,
                icon=folium.DivIcon(html=f"""<div style="transform: rotate({start_angle}deg);">&#10148;</div>""")
            ).add_to(m)
            folium.Marker(
                location=end,
                icon=folium.DivIcon(html=f"""<div style="transform: rotate({end_angle}deg);">&#10148;</div>""")
            ).add_to(m)

    legend_html_actual = """
    <div style="position: fixed;
                bottom: 50px; left: 50px; width: 150px; height: 150px;
                background-color: white; z-index:9999; font-size:14px;
                border:2px solid grey; padding: 10px;">
        <h4>Actual Cluster Points</h4>
    """
    for i in show_actual_clusters:
        legend_html_actual += f"""
        <i style="background: {actual_cluster_colors[i]}; width: 18px; height: 18px; float: left; margin-right: 8px;"></i> Cluster {i}<br>
        """
    legend_html_actual += "</div>"

    legend_html_predicted = """
    <div style="position: fixed;
                bottom: 50px; left: 250px; width: 150px; height: 150px;
                background-color: white; z-index:9999; font-size:14px;
                border:2px solid grey; padding: 10px;">
        <h4>Predicted Cluster Points</h4>
    """
    for i in show_predicted_clusters:
        legend_html_predicted += f"""
        <i style="background: {predicted_cluster_colors[i]}; width: 18px; height: 18px; float: left; margin-right: 8px;"></i> Cluster {i}<br>
        """
    legend_html_predicted += "</div>"

    m.get_root().html.add_child(folium.Element(legend_html_actual))
    m.get_root().html.add_child(folium.Element(legend_html_predicted))

    # Calculate and display average scores rounded to 5 decimal places
    avg_scores = actual_locations.groupby('Cluster')[['Size_Threat_Score', 'IntraCluster_Threat_Score', 'Overall_Threat_Score', 'Distance_Threat_Score']].mean().round(5).to_dict()
    avg_scores_html = """
    <div style="position: fixed;
                bottom: 50px; right: 50px; width: 250px; height: auto;
                background-color: white; z-index:9999; font-size:14px;
                border:2px solid grey; padding: 10px;">
        <h4>Anomalies</h4>
    """
    for cluster in avg_scores['Size_Threat_Score'].keys():
        threat_level = "Low"
        if avg_scores['Overall_Threat_Score'][cluster] > 0.8:
            threat_level = "High"
        elif 0.5 < avg_scores['Overall_Threat_Score'][cluster] <= 0.8:
            threat_level = "Medium"

        avg_scores_html += f"""
        <b>Cluster {cluster}:</b><br>
        <b>Threat Level: {threat_level}</b><br>
        Size Threat Score: {avg_scores['Size_Threat_Score'][cluster]:.5f}<br>
        IntraCluster Threat Score: {avg_scores['IntraCluster_Threat_Score'][cluster]:.5f}<br>
        Overall Threat Score: {avg_scores['Overall_Threat_Score'][cluster]:.5f}<br>
        Distance Threat Score: {avg_scores['Distance_Threat_Score'][cluster]:.5f}<br>
        <br>
        """
    avg_scores_html += "</div>"
    m.get_root().html.add_child(folium.Element(avg_scores_html))

    map_path = f'map_hour_{hour}.html'
    m.save(map_path)
    return map_path

def calculate_angle(lat1, lon1, lat2, lon2):
    angle = math.degrees(math.atan2(lat2 - lat1, lon2 - lon1))
    return angle

app = dash.Dash(__name__)

initial_hour = locations['Hour'].min()
initial_clusters = locations['Cluster'].unique()
initial_map_path = create_map(initial_hour, [], [], "All")

app.layout = html.Div([
    html.H1("Ship Locations Map"),
    html.Div(id='slider-container', children=[
        dcc.Input(
            id='hour-input',
            type='number',
            min=locations['Hour'].min(),
            max=locations['Hour'].max(),
            value=initial_hour,
            step=1
        ),
        dcc.Slider(
            id='hour-slider',
            min=locations['Hour'].min(),
            max=locations['Hour'].max(),
            value=initial_hour,
            marks={str(hour): str(hour) for hour in sorted(locations['Hour'].unique())},
            step=None,
            updatemode='drag'
        ),
        html.Button('▶️', id='play-button', style={'fontSize': 24}),
        html.Button('🔄', id='redo-button', style={'fontSize': 24}),
        dcc.Interval(
            id='interval-component',
            interval=1000,
            n_intervals=0,
            disabled=True
        ),
        html.Div(id='slider-output-container', style={'textAlign': 'center', 'marginTop': 20})
    ], style={'width': '90%', 'margin': 'auto'}),
    html.Div(style={'display': 'flex', 'justify-content': 'space-between'}, children=[
        html.Div(id='map-container', children=[
            html.Iframe(id='map', srcDoc=open(initial_map_path, 'r').read(), width='100%', height='900')
        ], style={'flex': '4'}),
        html.Div(style={'width': '10%', 'padding': '20px'}, children=[
            html.Label('Show Actual Points for Cluster:'),
            dcc.Checklist(
                id='show-actual-points',
                options=[{'label': f'Cluster {i}', 'value': i} for i in sorted(locations['Cluster'].unique())],
                value=[]
            ),
            html.Label('Show Predicted Points for Cluster:'),
            dcc.Checklist(
                id='show-predicted-points',
                options=[{'label': f'Cluster {i}', 'value': i} for i in sorted(locations['Cluster'].unique())],
                value=[]
            ),
            html.Label('Filter by Threat Level:'),
            dcc.Dropdown(
                id='threat-level-dropdown',
                options=[
                    {'label': 'All', 'value': 'All'},
                    {'label': 'Low', 'value': 'Low'},
                    {'label': 'Medium', 'value': 'Medium'},
                    {'label': 'High', 'value': 'High'}
                ],
                value='All',  # Default to "All" to show all anomalies initially
                placeholder="Select threat level"
            )
        ])
    ])
])

@app.callback(
    [Output('interval-component', 'disabled'),
     Output('play-button', 'children')],
    [Input('play-button', 'n_clicks')],
    [State('interval-component', 'disabled')]
)
def toggle_play(n_clicks, interval_disabled):
    if n_clicks:
        return (not interval_disabled, '⏸️' if interval_disabled else '▶️')
    return (interval_disabled, '▶️')

@app.callback(
    [Output('hour-slider', 'value'),
     Output('hour-input', 'value'),
     Output('map-container', 'children'),
     Output('slider-output-container', 'children')],
    [Input('interval-component', 'n_intervals'),
     Input('hour-slider', 'value'),
     Input('hour-input', 'value'),
     Input('redo-button', 'n_clicks'),
     Input('show-actual-points', 'value'),
     Input('show-predicted-points', 'value'),
     Input('threat-level-dropdown', 'value')],
    [State('interval-component', 'disabled')]
)
def update_map(n_intervals, slider_hour, input_hour, redo_clicks, show_actual_points, show_predicted_points, selected_threat_level, interval_disabled):
    ctx = dash.callback_context
    hours_list = sorted(locations['Hour'].unique())
    current_index = hours_list.index(slider_hour) if slider_hour in hours_list else 0

    if 'redo-button' in ctx.triggered[0]['prop_id']:
        hour = initial_hour
    elif not interval_disabled:
        next_index = (current_index + 1) % len(hours_list)
        hour = hours_list[next_index]
    else:
        hour = slider_hour if ctx.triggered[0]['prop_id'].startswith('hour-slider') else input_hour

    map_path = create_map(hour, show_actual_points, show_predicted_points, selected_threat_level)
    return (hour,
            hour,
            html.Iframe(id='map', srcDoc=open(map_path, 'r').read(), width='100%', height='900'),
            f'Selected Hour: {hour}')

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True)


<IPython.core.display.Javascript object>